# Quick Start
onetick-py is a pandas-like API for querying data (in particular, tick-by-tick [market data](https://www.onetick.com/market-data)) in [OneTick](https://www.onetick.com/time-series-database). This notebooks follows common business use cases to illustrate how they can be implemented with onetick-py.



In [12]:
# %%capture
# !pip install -U onetick.py

In [13]:
import onetick.py as otp
# define some parameters to re-use
start = otp.dt(2022, 3, 2, 10)
end = otp.dt(2022, 3, 2, 16)
emini = 'NQ\H22' #symbol for Nasdaq-100 E-mini future
otp.config['tz'] = 'EST5EDT' # set the timezone
print(otp.__version__)
import pandas as pd
import random
import itertools

1.14.41


# Retrieving data

Market data is time series data: i.e., each record or "tick" has a timestamp and ticks are ordered by timestamp. Our focus will be on trades, quotes, and order book data. We choose which type of data we look at by specifing the tick type: TRD for trades, QTE for quotes, and PRL for order books. The data is organized by the database, tick type, and security.

In [65]:
# create a query to retrieve data from the TRD tick type of the CME OneTick database
trd = otp.DataSource('CME', tick_type='TRD')
print(trd)

Observe that the output is a onetick-py DataSource object rather than the data. To retrieve the data into a pandas DataFrame, we invoke the `otp.run` method while providing it the time range and the symbols (instead `otp.run()`, we can use the shorthand `trd.to_df()` or just `trd()`).

In [66]:
trd_df = otp.run(trd, start=otp.dt(2022, 3, 2, 10), end=otp.dt(2022, 3, 2, 10, 1), symbols='NQ\H22')
print(trd_df)

                              Time           EXCH_TIME     PRICE  SIZE AGGRESSOR_SIDE TRADE_TYPE TRADE_PERIOD BOOK_TYPE   TRADE_ID   BUY_ORDER_ID  SELL_ORDER_ID        DELETED_TIME  TICK_STATUS  OMDSEQ
0    2022-03-02 10:00:00.000789627 1969-12-31 19:00:00  14074.75     1              S          -            -         0  236995996  6830860875515  6830860875655 1969-12-31 19:00:00            0       7
1    2022-03-02 10:00:00.000929635 1969-12-31 19:00:00  14075.75     1              B          -            -         0  236996008  6830860875705  6830860875691 1969-12-31 19:00:00            0      25
2    2022-03-02 10:00:00.000929635 1969-12-31 19:00:00  14076.00     1              B          -            -         0  236996009  6830860875705  6830860875569 1969-12-31 19:00:00            0      26
3    2022-03-02 10:00:00.000929635 1969-12-31 19:00:00  14076.00     1              B          -            -         0  236996009  6830860875705  6830860875694 1969-12-31 19:00:00            

We would rarely want to just retrieve market data (except for plotting it). Typical use cases involve computing analytics based on the tick-by-tick data. This is what onetick-py makes easy to do while keeping all of the processing in the highly optimized, parallelizable, and scalable tick management platform OneTick. The results of the analytics are returned into Python when the query is executed. We cover a few standard use cases next.

# Create minute bars from tick data
The code below prepares a query (a onetick-py DataSource object) to compute minute bars.

In [64]:
trd = otp.DataSource('CME', tick_type='TRD')
bars = trd.agg({'VOLUME': otp.agg.sum('SIZE'),
               'HIGH': otp.agg.max('PRICE'),
               'LOW': otp.agg.min('PRICE'),
               'OPEN': otp.agg.first('PRICE'),
                'COUNT': otp.agg.count(),
               'CLOSE': otp.agg.last('PRICE')},
               bucket_interval=60)
print(bars)

Only the bars, not the tick-by-tick trades used to compute them, are passed to Python when the query is executed.

In [17]:
bars_df = otp.run(bars, start=start, end=end, symbols=emini)
bars_df

,Time,VOLUME,HIGH,LOW,OPEN,COUNT,CLOSE
0,2022-03-02 10:01:00,1348,14085.00,14069.75,14074.75,1300,14077.75
1,2022-03-02 10:02:00,1558,14083.25,14066.25,14079.25,1485,14077.75
2,2022-03-02 10:03:00,1400,14091.75,14067.75,14077.50,1333,14084.75
3,2022-03-02 10:04:00,1721,14095.00,14071.25,14084.25,1594,14084.25
4,2022-03-02 10:05:00,1493,14098.50,14075.50,14084.00,1361,14096.75
...,...,...,...,...,...,...,...
355,2022-03-02 15:56:00,2499,14273.50,14250.25,14273.25,2343,14256.00
356,2022-03-02 15:57:00,1283,14259.75,14252.75,14256.25,1149,14254.25
357,2022-03-02 15:58:00,2970,14257.75,14240.00,14253.75,2511,14244.75
358,2022-03-02 15:59:00,3834,14246.50,14235.00,14244.50,3103,14239.75


Quick tip: you can display documentation for any object by typing "?" after it.

In [18]:
otp.math.exp?

Signature: otp.math.exp(value)
Docstring:
Compute the natural exponent of the `value`.

Parameters
----------
value: int, float, onetick.py.Operation, onetick.py.Column

Return
------
    onetick.py.Operation

Examples
--------
>>> data = otp.Tick(A=1)
>>> data['E'] = otp.math.exp(1)
>>> data.to_df()
        Time  A         E
0 2003-12-01  1  2.718282

See Also
--------
onetick.py.math.ln
File:      /usr/local/lib/python3.9/site-packages/onetick/py/math.py
Type:      function


# Filters
Filters are specified the same way as in pandas DataFrames with one caveat. Two objects are returned instead of one: one for to the ticks that pass the filter and the other for the ticks that don't (which you can store in `_` if you don't need it). We show an example of how to use both objects when we cover advanced concepts [below](#Filters:-using-both-if-and-else-branches).

In [67]:
trd = otp.DataSource('CME', tick_type='TRD')
trd, _ = trd[(trd['TRADE_PERIOD'] != 'A') & (trd['TRADE_TYPE'] != 'LEG')] # keep only non-leg and non-auction trades
trd = trd.first(5) # retrieve the first 5 records only
print(trd) # no data retrieval has been done yet
print(otp.run(trd, start=start, end=end, symbols=emini))

                           Time           EXCH_TIME     PRICE  SIZE AGGRESSOR_SIDE TRADE_TYPE TRADE_PERIOD BOOK_TYPE   TRADE_ID   BUY_ORDER_ID  SELL_ORDER_ID        DELETED_TIME  TICK_STATUS  OMDSEQ
0 2022-03-02 10:00:00.000789627 1969-12-31 19:00:00  14074.75     1              S          -            -         0  236995996  6830860875515  6830860875655 1969-12-31 19:00:00            0       7
1 2022-03-02 10:00:00.000929635 1969-12-31 19:00:00  14075.75     1              B          -            -         0  236996008  6830860875705  6830860875691 1969-12-31 19:00:00            0      25
2 2022-03-02 10:00:00.000929635 1969-12-31 19:00:00  14076.00     1              B          -            -         0  236996009  6830860875705  6830860875569 1969-12-31 19:00:00            0      26
3 2022-03-02 10:00:00.000929635 1969-12-31 19:00:00  14076.00     1              B          -            -         0  236996009  6830860875705  6830860875694 1969-12-31 19:00:00            0      27
4 202

<div class="alert alert-block alert-info">
Note: All code before `otp.run()` is executed in OneTick rather than in pandas/Python.
</div>

# Prevailing quote at the time of a trade
onetick-py provides some functions that are specific to time series analysis and do not exist in pandas. Joining two time series by time is one of such functions that makes finding the prevailing quote at the time of each trade a one-liner.

In [20]:
trd = otp.DataSource('CME', tick_type='TRD')
trd = trd[['PRICE', 'SIZE', 'TRADE_TYPE']] # just need these fields

qte = otp.DataSource('CME', tick_type='QTE')
qte['quote_time'] = qte['Time']
qte = qte[['ASK_PRICE', 'BID_PRICE', 'quote_time']] # just need these fields

# the first argument in the list (`trd` in this example) is the "leading" source
# all of the ticks from the leading source are enriched with the closest preceding 
# record from the non-leading source(s) (`qte` in this example).
enriched_trades = otp.join_by_time([trd, qte])

enriched_trades = enriched_trades.first(5)

otp.run(enriched_trades, start=start, end=end, symbols=emini)

,Time,PRICE,SIZE,TRADE_TYPE,ASK_PRICE,BID_PRICE,quote_time
0,2022-03-02 10:00:00.000789627,14074.75,1,-,14076.00,14074.75,2022-03-02 10:00:00.000788627
1,2022-03-02 10:00:00.000929635,14075.75,1,-,14075.75,14074.25,2022-03-02 10:00:00.000860873
2,2022-03-02 10:00:00.000929635,14076.00,1,-,14075.75,14074.25,2022-03-02 10:00:00.000860873
3,2022-03-02 10:00:00.000929635,14076.00,1,-,14075.75,14074.25,2022-03-02 10:00:00.000860873
4,2022-03-02 10:00:00.002409827,14075.75,1,-,14075.75,14075.00,2022-03-02 10:00:00.001767875


# Interval VWAP between order arrival and exit
Common TCA/BestEx use cases involve getting interval benchmarks such as market VWAP and market volume for each order. First we show how to compute the market volume and the VWAP for a single interval.

In [21]:
q = otp.DataSource('CME', tick_type='TRD')
q = q.agg({
'VOLUME': otp.agg.sum('SIZE'),
'VWAP': otp.agg.vwap('PRICE', 'SIZE')
})
otp.run(q, start=start, end=end, symbols=emini)

,Time,VOLUME,VWAP
0,2022-03-02 16:00:00,367803,14154.354845


Next we compute interval metrics for each of the orders arrival/exit intervals.

First we generate a simulated set of orders. Note that the orders/executions can come from any source (e.g., CSV file, FIX log file, database, Python object, OneTick database, etc).

In [22]:
num_orders = 20
arrival_time = [otp.dt(2022, 3, 2, 10) + otp.Milli(x*500) for x in range(0, num_orders)]
exit_time = [x + otp.Milli(random.randint(100,10000)) for x in arrival_time]
symbols = ['NQ\H22', 'ES\H22']

order_df = pd.DataFrame({
    'arrival_time': arrival_time, 
    'exit_time': exit_time, 
    'ID': [x for x in range(0, num_orders)],
    'SYMBOL_NAME': [random.choice(symbols) for x in range(0, num_orders)],
})
order_df

,arrival_time,exit_time,ID,SYMBOL_NAME
0,2022-03-02 10:00:00,2022-03-02 10:00:04.790000,0,NQ\H22
1,2022-03-02 10:00:00.500000,2022-03-02 10:00:05.839000,1,NQ\H22
2,2022-03-02 10:00:01,2022-03-02 10:00:10.150000,2,ES\H22
3,2022-03-02 10:00:01.500000,2022-03-02 10:00:08.854000,3,NQ\H22
4,2022-03-02 10:00:02,2022-03-02 10:00:08.729000,4,ES\H22
5,2022-03-02 10:00:02.500000,2022-03-02 10:00:09.730000,5,NQ\H22
6,2022-03-02 10:00:03,2022-03-02 10:00:04.286000,6,ES\H22
7,2022-03-02 10:00:03.500000,2022-03-02 10:00:08.825000,7,NQ\H22
8,2022-03-02 10:00:04,2022-03-02 10:00:07.576000,8,ES\H22
9,2022-03-02 10:00:04.500000,2022-03-02 10:00:06.878000,9,ES\H22


Set up the fields that will enable us to query market trades for symbol and the arrival/exit interval of each order in parallel.

In [23]:
# convert the DataFrame with orders into a onetick-py object to enable calls to onetick-py functions
order = otp.Ticks(order_df.to_dict(orient='list'))

order['_PARAM_START_TIME_NANOS'] = order['arrival_time'] # this will be the start time of the query
order['_PARAM_END_TIME_NANOS'] = order['exit_time'] # this will be the end time of the query
# another relevant field is SYMBOL_NAME (already specified) which will be used as the symbol for the query

We can now define a function to compute VWAP and volume for each order. The function takes an argument (`symbol`) that is passed automatically when and the order specific info we defined above.

In [24]:
def market_volume_and_vwap(symbol):
    trd = otp.DataSource('CME', tick_type='TRD')
    vol_vwap = trd.agg({
    'VOLUME': otp.agg.sum('SIZE'),
    'VWAP': otp.agg.vwap('PRICE', 'SIZE')
    }, bucket_time='start')
    vol_vwap['ID'] = symbol['ID']
    vol_vwap['exit_time'] = symbol['exit_time']
    vol_vwap['sym'] = symbol.name # SYMBOL_NAME is stored in a special attribute '.name'
    
    return vol_vwap

`market_volume_and_vwap` can now be executed for each order. The computation is parallelized by symbol.

In [25]:
res = otp.run(market_volume_and_vwap, symbols=order, concurrency=8, batch_size=100)

The result is a `dict` keyed by symbol.

In [26]:
for s in res.keys():
    print(pd.concat(res[s]))

                     Time  VOLUME          VWAP  ID               exit_time     sym
0 2022-03-02 05:00:00.000      12  14092.958333   0 2022-03-02 05:00:04.790  NQ\H22
0 2022-03-02 05:00:00.500      12  14092.958333   1 2022-03-02 05:00:05.839  NQ\H22
0 2022-03-02 05:00:01.500      21  14093.369048   3 2022-03-02 05:00:08.854  NQ\H22
0 2022-03-02 05:00:02.500      19  14093.657895   5 2022-03-02 05:00:09.730  NQ\H22
0 2022-03-02 05:00:03.500      16  14093.812500   7 2022-03-02 05:00:08.825  NQ\H22
0 2022-03-02 05:00:05.500      18  14093.791667  11 2022-03-02 05:00:13.914  NQ\H22
0 2022-03-02 05:00:06.000      10  14093.925000  12 2022-03-02 05:00:06.450  NQ\H22
0 2022-03-02 05:00:07.000       4  14093.812500  14 2022-03-02 05:00:08.672  NQ\H22
0 2022-03-02 05:00:07.500       5  14093.850000  15 2022-03-02 05:00:12.303  NQ\H22
0 2022-03-02 05:00:08.000       3  14094.000000  16 2022-03-02 05:00:08.635  NQ\H22
0 2022-03-02 05:00:08.500       2  14093.625000  17 2022-03-02 05:00:13.699 

# Point-in-time benchmarks: BBO at a given time

Another common use case is find the prevailing quote (best bid/ask) at the time of an execution or at different time intervals (markouts) before/after the execution.

The following snippet find the prevailing quote at a give time.

In [27]:
q = otp.DataSource('CME', tick_type='QTE', back_to_first_tick=600, keep_first_tick_timestamp='BBO_TIME')
q = q[['ASK_PRICE','BID_PRICE', 'BBO_TIME']]
time = otp.dt(2022,3,1,19,25,48,123456)
otp.run(q, start=time, end=time, symbols=emini)

,Time,ASK_PRICE,BID_PRICE,BBO_TIME
0,2022-03-01 19:25:48.123456,14058.0,14056.25,2022-03-01 19:25:47.266282997


# Point-in-time benchmarks: BBO at different offsets (markouts)

We'll find the BBO at different offets in seconds around the time of each execution.

We create a simulated set of executions as we did above for orders.

In [28]:
num_execs = 5
exec_time = [otp.dt(2022, 3, 2, 10) + otp.Milli(x*500) for x in range(0, num_execs)]
symbols = ['NQ\H22', 'ES\H22']

exec_df = pd.DataFrame({
    'exex_time': exec_time, 
    'ID': [x for x in range(0, num_execs)],
    'SYMBOL_NAME': [random.choice(symbols) for x in range(0, num_execs)],
})
exec_df

,exex_time,ID,SYMBOL_NAME
0,2022-03-02 10:00:00,0,ES\H22
1,2022-03-02 10:00:00.500000,1,ES\H22
2,2022-03-02 10:00:01,2,ES\H22
3,2022-03-02 10:00:01.500000,3,ES\H22
4,2022-03-02 10:00:02,4,NQ\H22


Create a separate record for each markout with the corresponding timestamp.

In [29]:
markouts = [-.5, 0, .5, 1, 5, 10] # in seconds: e.g., .5 means half a second after the execution

executions_with_markouts_df = pd.DataFrame({
 'markout_time': [x + otp.Milli(int(m*1000)) for x in exec_time for m in markouts], 
 'ID': [x for x in range(0, num_execs) for m in markouts],
 'SYMBOL_NAME': list(itertools.chain(*[[random.choice(symbols)] * len(markouts) for x in range(0, num_execs)])),
 'markout': [m for x in range(0, num_execs) for m in markouts],
})

executions_with_markouts_df

,markout_time,ID,SYMBOL_NAME,markout
0,2022-03-02 09:59:59.500000,0,ES\H22,-0.5
1,2022-03-02 10:00:00,0,ES\H22,0.0
2,2022-03-02 10:00:00.500000,0,ES\H22,0.5
3,2022-03-02 10:00:01,0,ES\H22,1.0
4,2022-03-02 10:00:05,0,ES\H22,5.0
5,2022-03-02 10:00:10,0,ES\H22,10.0
6,2022-03-02 10:00:00,1,NQ\H22,-0.5
7,2022-03-02 10:00:00.500000,1,NQ\H22,0.0
8,2022-03-02 10:00:01,1,NQ\H22,0.5
9,2022-03-02 10:00:01.500000,1,NQ\H22,1.0


Convert DataFrame to a `onetick-py` object and specify start/end times. Note that for the markouts query, the start and end times are the same as we'll look backwards from the markout timestamp.

In [30]:
executions_with_markouts = otp.Ticks(executions_with_markouts_df.to_dict(orient='list'))
executions_with_markouts['_PARAM_START_TIME_NANOS'] = executions_with_markouts['markout_time']
executions_with_markouts['_PARAM_END_TIME_NANOS'] = executions_with_markouts['markout_time']

Define a function that looks back from the start of the query until it finds the first tick. Execute the function for each execution/markout.

In [31]:
def bbo_at_time(symbol):
    q = otp.DataSource('CME', tick_type='QTE', back_to_first_tick=600, keep_first_tick_timestamp='BBO_TIME')
    q = q[['ASK_PRICE','BID_PRICE', 'BBO_TIME']]
    q['ID'] = symbol['ID']
    q['markout'] = symbol['markout']
    q['sym'] = symbol.name
    return q

res = otp.run(bbo_at_time, symbols=executions_with_markouts, concurrency=8, batch_size=100)

for s in res.keys():
    print(pd.concat(res[s]))

                     Time  ASK_PRICE  BID_PRICE                      BBO_TIME  ID  markout     sym
0 2022-03-02 04:59:59.500    4333.75    4333.25 2022-03-02 04:59:59.499993709   0     -0.5  ES\H22
0 2022-03-02 05:00:00.000    4333.25    4333.00 2022-03-02 04:59:59.930848677   0      0.0  ES\H22
0 2022-03-02 05:00:00.500    4333.75    4333.25 2022-03-02 05:00:00.494756933   0      0.5  ES\H22
0 2022-03-02 05:00:01.000    4334.00    4333.50 2022-03-02 05:00:00.931358191   0      1.0  ES\H22
0 2022-03-02 05:00:05.000    4333.50    4333.25 2022-03-02 05:00:04.631332237   0      5.0  ES\H22
0 2022-03-02 05:00:10.000    4333.75    4333.25 2022-03-02 05:00:09.936250013   0     10.0  ES\H22
0 2022-03-02 05:00:01.000    4334.00    4333.50 2022-03-02 05:00:00.931358191   3     -0.5  ES\H22
0 2022-03-02 05:00:01.500    4334.00    4333.50 2022-03-02 05:00:01.481454199   3      0.0  ES\H22
0 2022-03-02 05:00:02.000    4333.50    4333.00 2022-03-02 05:00:01.982271033   3      0.5  ES\H22
0 2022-03-

# Upticks/Downticks
You can mark each trade as an uptick or downtick.

In [32]:
trd = otp.DataSource('CME', tick_type='TRD')

def uptick(row):
    if row['PRICE'] == otp.nan or row['PRICE'][-1] == otp.nan:
        return otp.nan
    if row['PRICE'] > row['PRICE'][-1]:
        return 1
    elif row['PRICE'] < row['PRICE'][-1]:
        return -1
    else:
        return 0

trd['UPTICK'] = trd.apply(uptick)
trd = trd[['PRICE', 'UPTICK']]
trd = trd.first(5)
otp.run(trd, start=start, end=end, symbols=emini)

,Time,PRICE,UPTICK
0,2022-03-02 10:00:00.000789627,14074.75,NaN
1,2022-03-02 10:00:00.000929635,14075.75,1.0
2,2022-03-02 10:00:00.000929635,14076.00,1.0
3,2022-03-02 10:00:00.000929635,14076.00,0.0
4,2022-03-02 10:00:00.002409827,14075.75,-1.0


# Symbol Operations

## Quering multiple symbols

### A separate data series for each symbol

In [33]:
symbols=['AAPL','MSFT'] # sybmols can be specified as a list
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
trd = trd[['PRICE','SIZE','EXCHANGE']]
trd = trd.first(5)
trd_df = otp.run(trd, symbols=symbols, concurrency=8, start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))
trd_df

{'MSFT':                            Time   PRICE  SIZE EXCHANGE
 0 2022-03-01 10:00:00.003577531  298.53    50        Q
 1 2022-03-01 10:00:00.012147126  298.53    60        K
 2 2022-03-01 10:00:00.012251683  298.53    40        K
 3 2022-03-01 10:00:00.012524905  298.54    27        P
 4 2022-03-01 10:00:00.012602204  298.54     2        P,
 'AAPL':                            Time   PRICE  SIZE EXCHANGE
 0 2022-03-01 10:00:00.003529175  165.75   100        Q
 1 2022-03-01 10:00:00.003530928  165.75   100        Q
 2 2022-03-01 10:00:00.003532720  165.75   100        Q
 3 2022-03-01 10:00:00.003676379  165.75   100        Z
 4 2022-03-01 10:00:00.004032611  165.75   104        H}

### Ticks from all symbols merged into a single data series

In [34]:
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
trd = trd[['PRICE','SIZE','EXCHANGE','COND']]
trd = otp.merge([trd], symbols=['AAPL','MSFT'], identify_input_ts=True)
trd = trd.first(5)
otp.run(trd, start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))

,Time,PRICE,SIZE,EXCHANGE,COND,SYMBOL_NAME,TICK_TYPE
0,2022-03-01 10:00:00.003529175,165.75,100,Q,@,AAPL,TRD
1,2022-03-01 10:00:00.003530928,165.75,100,Q,@,AAPL,TRD
2,2022-03-01 10:00:00.003532720,165.75,100,Q,@,AAPL,TRD
3,2022-03-01 10:00:00.003577531,298.53,50,Q,@ I,MSFT,TRD
4,2022-03-01 10:00:00.003676379,165.75,100,Z,@,AAPL,TRD


Symbols can come from different databases, in which case, the database name and '::' would appear
before the symbol name. The schema must be speficied explicitly in this case.

In [35]:
trd = otp.DataSource(tick_type='TRD')
trd = trd.table(PRICE=float, SIZE=int, EXCH_TIME=otp.nsectime, EXCHANGE=otp.string[8])
trd = otp.merge([trd], symbols=['IDC1075::FL','MS113::FL.113'], identify_input_ts=True)
trd = trd.first(5)
otp.run(trd, start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))

,Time,PRICE,SIZE,EXCH_TIME,EXCHANGE,SYMBOL_NAME,TICK_TYPE
0,2022-03-01 10:01:27.612,2.83,2000,2022-03-01 10:01:27.582372000,ALP,IDC1075::FL,TRD
1,2022-03-01 10:02:00.803,2.83,100,2022-03-01 10:02:00.803000000,CHI,IDC1075::FL,TRD
2,2022-03-01 10:02:00.803,2.83,600,2022-03-01 10:02:00.803162000,CDX,IDC1075::FL,TRD
3,2022-03-01 10:02:00.914,2.83,600,2022-03-01 10:02:00.803162203,,MS113::FL.113,TRD
4,2022-03-01 10:07:05.436,2.84,3,2022-03-01 10:07:05.436496000,TCM,IDC1075::FL,TRD


## Get all symbols from a database

In [36]:
symbols = otp.Symbols(db='NYSE_TAQ', date=otp.dt(2022, 3, 1))
otp.run(symbols)

,Time,SYMBOL_NAME
0,2022-03-01,A
1,2022-03-01,AA
2,2022-03-01,AAA
3,2022-03-01,AAAU
4,2022-03-01,AAC
...,...,...
12041,2022-03-01,ZXZZT
12042,2022-03-01,ZY
12043,2022-03-01,ZYME
12044,2022-03-01,ZYNE


## Get all symbols that match a pattern

In [37]:
symbols = otp.Symbols(db=['NYSE_TAQ'], date=otp.dt(2022, 3, 1), pattern='BBA%')
otp.run(symbols)

,Time,SYMBOL_NAME
0,2022-03-01,BBAI
1,2022-03-01,BBAIWS
2,2022-03-01,BBAR
3,2022-03-01,BBAX


## Count the number of trades per symbol

In [38]:
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
count = trd.agg({'COUNT': otp.agg.count()})
otp.run(count, concurrency=8, start=otp.dt(2022,3,1), end=otp.dt(2022,3,2), symbols=otp.Symbols(db='NYSE_TAQ', pattern='BBA%'))

{'BBAIWS':         Time  COUNT
 0 2022-03-02    137,
 'BBAX':         Time  COUNT
 0 2022-03-02   1115,
 'BBAI':         Time  COUNT
 0 2022-03-02   1063,
 'BBAR':         Time  COUNT
 0 2022-03-02   3281}

## Count the number of trades across symbols

In [39]:
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
count = trd.agg({'COUNT': otp.agg.count()})
count = otp.merge([count], symbols=otp.Symbols(db='NYSE_TAQ'), presort=True, identify_input_ts=True)
total_count = count.agg({'COUNT': otp.agg.sum('COUNT')})
otp.run(total_count, concurrency=8, start=otp.dt(2022,3,1), end=otp.dt(2022,3,2))

,Time,COUNT
0,2022-03-02,102893787


## Find the most traded symbols and query them (`Multi-stage queries`)

In [40]:
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
count = trd.agg({'COUNT': otp.agg.count()})
count = otp.merge([count], symbols=otp.Symbols(db='NYSE_TAQ'), presort=True, identify_input_ts=True)
most_traded = count.high('COUNT', n=5)
otp.run(most_traded, concurrency=8, start=otp.dt(2022,3,1), end=otp.dt(2022,3,2))

,Time,COUNT,SYMBOL_NAME,TICK_TYPE
0,2022-03-02,1422076,SPY,TRD
1,2022-03-02,883708,TQQQ,TRD
2,2022-03-02,844847,AMD,TRD
3,2022-03-02,829879,SQQQ,TRD
4,2022-03-02,763562,QQQ,TRD


Now that we have a query that finds the most traded symbols, we can use it to provide these symbols for querying

In [41]:
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
count = trd.agg({'COUNT': otp.agg.count()})
count = otp.merge([count], symbols=otp.Symbols(db='NYSE_TAQ'), presort=True, identify_input_ts=True)
most_traded = count.high('COUNT', n=5)

trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
trd = trd.first(5)
otp.run(trd, concurrency=8, symbols=most_traded, start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))

{'AAPL':                            Time EXCHANGE  COND STOP_STOCK SOURCE TRF TTE TICKER   PRICE        DELETED_TIME  TICK_STATUS  SIZE  CORR  SEQ_NUM TRADE_ID              PARTICIPANT_TIME            TRF_TIME  OMDSEQ
 0 2022-03-01 10:00:00.003529175        Q  @                    N       0   AAPL  165.75 1969-12-31 19:00:00            0   100     0  1009136    29549 2022-03-01 10:00:00.003512815 1969-12-31 19:00:00       0
 1 2022-03-01 10:00:00.003530928        Q  @                    N       0   AAPL  165.75 1969-12-31 19:00:00            0   100     0  1009137    29550 2022-03-01 10:00:00.003512815 1969-12-31 19:00:00       1
 2 2022-03-01 10:00:00.003532720        Q  @                    N       0   AAPL  165.75 1969-12-31 19:00:00            0   100     0  1009138    29551 2022-03-01 10:00:00.003512815 1969-12-31 19:00:00       2
 3 2022-03-01 10:00:00.003676379        Z  @                    N       0   AAPL  165.75 1969-12-31 19:00:00            0   100     0  1009139    13157 

# Order book analytics

`onetick-py` offers functions for analyzing tick-by-tick order book. There are three representations of an order book. We'll show top 3 levels only for ease of exposition.

A book can be displayed with a tick per level per side. We refer to a level in the book as a 'price level' or 'prl'.

In [42]:
snapshot_time=start
prl = otp.ObSnapshot(db='CME', tick_type='PRL_FULL', max_levels=3) 
otp.run(prl, symbols=emini, start=snapshot_time, end=snapshot_time)

,Time,PRICE,UPDATE_TIME,SIZE,LEVEL,BUY_SELL_FLAG
0,2022-03-02 10:00:00,14076.00,2022-03-02 09:59:59.992707035,1,1,1
1,2022-03-02 10:00:00,14076.25,2022-03-02 09:59:59.953516579,2,2,1
2,2022-03-02 10:00:00,14076.50,2022-03-02 09:59:59.893691735,4,3,1
3,2022-03-02 10:00:00,14075.25,2022-03-02 09:59:59.992733435,1,1,0
4,2022-03-02 10:00:00,14075.00,2022-03-02 09:59:59.954720897,2,2,0
5,2022-03-02 10:00:00,14074.75,2022-03-02 09:59:59.948987231,2,3,0


Alternatively, a book can show a tick per level with both ask and bid price/size info.

In [43]:
snapshot_time=start
prl = otp.ObSnapshotWide(db='CME', tick_type='PRL_FULL', max_levels=3)   
otp.run(prl, symbols=emini, start=snapshot_time, end=snapshot_time)

,Time,BID_PRICE,BID_UPDATE_TIME,BID_SIZE,ASK_PRICE,ASK_UPDATE_TIME,ASK_SIZE,LEVEL
0,2022-03-02 10:00:00,14075.25,2022-03-02 09:59:59.992733435,1,14076.00,2022-03-02 09:59:59.992707035,1,1
1,2022-03-02 10:00:00,14075.00,2022-03-02 09:59:59.954720897,2,14076.25,2022-03-02 09:59:59.953516579,2,2
2,2022-03-02 10:00:00,14074.75,2022-03-02 09:59:59.948987231,2,14076.50,2022-03-02 09:59:59.893691735,4,3


Finally, all levels can be displayed in one tick.

In [68]:
snapshot_time=start
prl = otp.ObSnapshotFlat(db='CME', tick_type='PRL_FULL', max_levels=3) 
print(otp.run(prl, symbols=emini, start=snapshot_time, end=snapshot_time))

                 Time  BID_PRICE1              BID_UPDATE_TIME1  BID_SIZE1  ASK_PRICE1              ASK_UPDATE_TIME1  ASK_SIZE1  BID_PRICE2              BID_UPDATE_TIME2  BID_SIZE2  ASK_PRICE2              ASK_UPDATE_TIME2  ASK_SIZE2  BID_PRICE3              BID_UPDATE_TIME3  BID_SIZE3  ASK_PRICE3              ASK_UPDATE_TIME3  ASK_SIZE3
0 2022-03-02 10:00:00    14075.25 2022-03-02 09:59:59.992733435          1     14076.0 2022-03-02 09:59:59.992707035          1     14075.0 2022-03-02 09:59:59.954720897          2    14076.25 2022-03-02 09:59:59.953516579          2    14074.75 2022-03-02 09:59:59.948987231          2     14076.5 2022-03-02 09:59:59.893691735          4


We can output the book (in any of the three representation) on every change to price/size at any of the levels.

In [69]:
prl = otp.ObSnapshotFlat(db='CME', tick_type='PRL_FULL', max_levels=3, running=True)
prl = prl.drop(r".+TIME\d")
print(otp.run(prl, symbols=emini, start=start,  end=start+otp.Milli(100)))

                             Time  BID_PRICE1  BID_SIZE1  ASK_PRICE1  ASK_SIZE1  BID_PRICE2  BID_SIZE2  ASK_PRICE2  ASK_SIZE2  BID_PRICE3  BID_SIZE3  ASK_PRICE3  ASK_SIZE3
0   2022-03-02 10:00:00.000000000    14075.25          1    14076.00          1    14075.00          2    14076.25          2    14074.75          2    14076.50          4
1   2022-03-02 10:00:00.000788455    14075.00          2    14076.00          1    14074.75          2    14076.25          2    14074.50          2    14076.50          4
2   2022-03-02 10:00:00.000788627    14074.75          2    14076.00          1    14074.50          2    14076.25          2    14074.25          2    14076.50          4
3   2022-03-02 10:00:00.000789627    14074.50          2    14076.00          1    14074.25          2    14076.25          2    14074.00          2    14076.50          4
4   2022-03-02 10:00:00.000790993    14074.50          1    14076.00          1    14074.25          2    14076.25          2    14074.00   

The `ObSnapshot` method doesn't require specifying `max_levels`. The entire book is returned when the parameter is not specified.

In [46]:
snapshot_time=start
prl = otp.ObSnapshot(db='CME', tick_type='PRL_FULL') 
otp.run(prl, symbols=emini, start=snapshot_time, end=snapshot_time)

,Time,PRICE,UPDATE_TIME,SIZE,LEVEL,BUY_SELL_FLAG
0,2022-03-02 10:00:00,14076.00,2022-03-02 09:59:59.992707035,1,1,1
1,2022-03-02 10:00:00,14076.25,2022-03-02 09:59:59.953516579,2,2,1
2,2022-03-02 10:00:00,14076.50,2022-03-02 09:59:59.893691735,4,3,1
3,2022-03-02 10:00:00,14076.75,2022-03-02 09:59:59.893054207,2,4,1
4,2022-03-02 10:00:00,14077.00,2022-03-02 09:59:59.334717127,2,5,1
...,...,...,...,...,...,...
1628,2022-03-02 10:00:00,6490.00,2022-03-01 17:59:59.999000000,3,809,0
1629,2022-03-02 10:00:00,1586.00,2022-03-01 17:59:59.999000000,1,810,0
1630,2022-03-02 10:00:00,786.50,2022-03-01 17:59:59.999000000,1,811,0
1631,2022-03-02 10:00:00,200.00,2022-03-01 17:59:59.999000000,1,812,0


## Book Imbalance

Let's find the time weighted book imbalance. The imbalance at a given time is defined as the sum of the bid sizes at the top x levels minus the sum of the ask sizes at the top x levels divided by the sum of these two terms: the values close to 1 mean the book is much heavier on the bid side, close to -1 -- on the ask side, equal to zero means the sizes are the same.

In [47]:
x = 3

prl = otp.ObSnapshotWide(db='CME', tick_type='PRL_FULL', max_levels=x, running=True)
prls_df = otp.run(prl, symbols=emini, start=start, end=start+otp.Milli(100))
print(prls_df.head(7))

prl = prl.agg({'ask_vol': otp.agg.sum('ASK_SIZE'), 'bid_vol': otp.agg.sum('BID_SIZE')}, bucket_units='ticks', bucket_interval=x)
prl['imb'] = (prl['bid_vol'] - prl['ask_vol']) / (prl['bid_vol'] + prl['ask_vol'])
prls_df = otp.run(prl, symbols=emini, start=start, end=start+otp.Milli(100))
print(prls_df.head())

imb_stats = prl.agg({
    'tw_imb': otp.agg.tw_average('imb'),
    'mean':   otp.agg.average('imb'),
    'stdev':  otp.agg.stddev('imb'),
})
print(otp.run(imb_stats, symbols=emini, start=start, end=start+otp.Milli(100)))



                           Time  BID_PRICE               BID_UPDATE_TIME  BID_SIZE  ASK_PRICE               ASK_UPDATE_TIME  ASK_SIZE  LEVEL
0 2022-03-02 10:00:00.000000000   14075.25 2022-03-02 09:59:59.992733435         1   14076.00 2022-03-02 09:59:59.992707035         1      1
1 2022-03-02 10:00:00.000000000   14075.00 2022-03-02 09:59:59.954720897         2   14076.25 2022-03-02 09:59:59.953516579         2      2
2 2022-03-02 10:00:00.000000000   14074.75 2022-03-02 09:59:59.948987231         2   14076.50 2022-03-02 09:59:59.893691735         4      3
3 2022-03-02 10:00:00.000788455   14075.00 2022-03-02 09:59:59.954720897         2   14076.00 2022-03-02 09:59:59.992707035         1      1
4 2022-03-02 10:00:00.000788455   14074.75 2022-03-02 09:59:59.948987231         2   14076.25 2022-03-02 09:59:59.953516579         2      2
5 2022-03-02 10:00:00.000788455   14074.50 2022-03-02 09:59:59.893306297         2   14076.50 2022-03-02 09:59:59.893691735         4      3
6 2022-03-02 

## Book sweep

There are two version of book sweep: by price and by quantity. Book sweep by price, take a price as an input and returns the total quatity available at that price or better. Book sweep by quantity, takes a quantity as an input and returns the VWAP if the quantity were executed immediately.

In [48]:
def side_to_direction(side):
    return 1 if side == 'ASK' else -1

def sweep_by_price(side, price):
    prl = otp.ObSnapshot(db='CME', tick_type='PRL_FULL', side=side)
    direction = side_to_direction(side)
    prl, _ = prl[direction * prl['PRICE'] <= direction * price]
    prl = prl.agg({'total_qty': otp.agg.sum('SIZE')})
    return otp.run(prl, symbols=emini, start=start, end=start)

print(sweep_by_price('BID', 14075))
print(sweep_by_price('ASK', 14077))

                 Time  total_qty
0 2022-03-02 10:00:00          3
                 Time  total_qty
0 2022-03-02 10:00:00         11


In [49]:
def sweep_by_qty(side, qty):
    prl = otp.ObSnapshot(db='CME', tick_type='PRL_FULL', side=side)
    prl = prl.agg({'total_qty': otp.agg.sum('SIZE')}, running=True, all_fields=True)
    direction = side_to_direction(side)
    prl, _ = prl[prl['total_qty'] - prl['SIZE'] < qty]
    # update the SIZE in the last tick only so that total_qty is exactly qty
    prl['SIZE'] = prl.apply(lambda tick: prl['SIZE'] - (prl['total_qty'] - qty) if prl['total_qty'] > qty else prl['SIZE'])
    prl = prl.agg({'VWAP': otp.agg.vwap('PRICE', 'SIZE')})
    return otp.run(prl, symbols=emini, start=start, end=start)
print(sweep_by_qty('BID', 10))
print(sweep_by_qty('ASK', 10))

                 Time       VWAP
0 2022-03-02 10:00:00  14074.625
                 Time     VWAP
0 2022-03-02 10:00:00  14076.5


## Market By Order
Order Book data may be annotated with 'key' fields lets you break down the book by each value of the 'key' fields. For example, a book could by keyed by market participant ID, allowing you to see the book with the orders of a given market participant only. Some exchanges provide 'market-by-order' data where the book is keyed by order id. Set `show_full_detail` to `True` to see the book broken down to the most granular level. The example below is a market-by-order book.

In [70]:
prl = otp.ObSnapshot('CME', tick_type='PRL_FULL', side='BID', show_full_detail=True)
prl = prl.first(5)
print(otp.run(prl, symbols=emini, start=start, end=start))

                 Time       ORDER_ID  BUY_SELL_FLAG ORDER_TYPE     PRICE  SIZE  TIME_PRIORITY RECORD_TYPE        DELETED_TIME  TICK_STATUS  OMDSEQ  LEVEL                   UPDATE_TIME
0 2022-03-02 10:00:00  6830860875633              0          L  14075.25     1    43132163778           R 1969-12-31 19:00:00            0       3      1 2022-03-02 09:59:59.992733435
1 2022-03-02 10:00:00  6830860875525              0          L  14075.00     1    43132163642           R 1969-12-31 19:00:00            0      11      2 2022-03-02 09:59:59.949569201
2 2022-03-02 10:00:00  6830860875517              0          L  14075.00     1    43132163634           R 1969-12-31 19:00:00            0       3      2 2022-03-02 09:59:59.949264581
3 2022-03-02 10:00:00  6830860875515              0          L  14074.75     1    43132163632           R 1969-12-31 19:00:00            0       0      3 2022-03-02 09:59:59.948987231
4 2022-03-02 10:00:00  6830860875480              0          L  14074.75     1  

Market-by-order data can be used to analyze/validate the priority mechanism used by the exchange.

In [51]:
prl = otp.ObSnapshot('CME', tick_type='PRL_FULL', side='BID', show_full_detail=True)

"""
ORDER_TYPE:
L = Limit order
I = Implied order

Implied liquidity doesn’t have priority as it's always last to execute at any price level. 
It also doesn’t have an order ID, so the IDs that you see in the db are synthetic 
(consisting of 1 or 2 for the 1st/2nd implied level, and E/F for the buy/sell side respectively).

In order to rank the orders within a given price point by priority, you need to sort first by ORDER_TYPE (“L” comes before “I”),
then by TIME_PRIORITY (lowest value comes first).
"""
prl = prl.sort(['LEVEL','ORDER_TYPE', 'TIME_PRIORITY'],ascending=[True,False, True])
orders = otp.run(prl, symbols=emini, start=start, end=start);
orders = orders[['ORDER_ID', 'PRICE', 'LEVEL', 'TIME_PRIORITY','SIZE', 'BUY_SELL_FLAG', 'ORDER_TYPE', 'RECORD_TYPE', 'UPDATE_TIME', 'TICK_STATUS']]
orders.head()

,ORDER_ID,PRICE,LEVEL,TIME_PRIORITY,SIZE,BUY_SELL_FLAG,ORDER_TYPE,RECORD_TYPE,UPDATE_TIME,TICK_STATUS
0,6830860875633,14075.25,1,43132163778,1,0,L,R,2022-03-02 09:59:59.992733435,0
1,6830860875517,14075.00,2,43132163634,1,0,L,R,2022-03-02 09:59:59.949264581,0
2,6830860875525,14075.00,2,43132163642,1,0,L,R,2022-03-02 09:59:59.949569201,0
3,6830860875480,14074.75,3,43132163583,1,0,L,R,2022-03-02 09:59:59.910612819,0
4,6830860875515,14074.75,3,43132163632,1,0,L,R,2022-03-02 09:59:59.948987231,0


# Best practices

## The golden rule of otp: call `otp.run` as late as possible
Processing before the `otp.run` call is good because 
- it takes place in OneTick which is optimized for time series processing
- it is parallelized by symbol and query (e.g., when "join with query" is called)
- it takes place in a scalable environment

Processing after the `otp.run` call is bad because
- it takes place in python which is slow
- it takes place in the notebook which is limited by a fixed and normally small instance size where the notebook runs

Bad:

In [52]:
trd = otp.DataSource('CME', tick_type='TRD')
trd_df = otp.run(trd, start=start, end=end, symbols=emini)
trd_df['SIZE'].agg('sum')

367803

Good:

In [53]:
trd = otp.DataSource('CME', tick_type='TRD')

trd = trd.agg({'VOLUME': otp.agg.sum(trd['SIZE'])})

otp.run(trd, start=start, end=end, symbols=emini)

,Time,VOLUME
0,2022-03-02 16:00:00,367803


Bad:

In [71]:
trd = otp.DataSource('CME', tick_type='TRD')
trd_df = otp.run(trd, start=start, end=end, symbols=emini)
print(trd_df[trd_df['TRADE_TYPE'] == 'LEG'].head())

                              Time           EXCH_TIME  PRICE  SIZE AGGRESSOR_SIDE TRADE_TYPE TRADE_PERIOD BOOK_TYPE TRADE_ID BUY_ORDER_ID SELL_ORDER_ID        DELETED_TIME  TICK_STATUS  OMDSEQ
271  2022-03-02 10:00:07.592961899 1969-12-31 19:00:00    NaN     1                       LEG            -         1                                     1969-12-31 19:00:00            0       9
272  2022-03-02 10:00:07.593666573 1969-12-31 19:00:00    NaN     2                       LEG            -         1                                     1969-12-31 19:00:00            0       0
3114 2022-03-02 10:02:20.046438331 1969-12-31 19:00:00    NaN     3                       LEG            -         1                                     1969-12-31 19:00:00            0       0
4753 2022-03-02 10:03:15.584272405 1969-12-31 19:00:00    NaN     1                       LEG            -         1                                     1969-12-31 19:00:00            0       3
5328 2022-03-02 10:03:35.61218

Good:

In [72]:
trd = otp.DataSource('CME', tick_type='TRD')
leg, notleg = trd[trd['TRADE_TYPE'] == 'LEG']
leg = leg.first(5)
print(otp.run(leg, start=start, end=end, symbols=emini))

                           Time           EXCH_TIME  PRICE  SIZE AGGRESSOR_SIDE TRADE_TYPE TRADE_PERIOD BOOK_TYPE TRADE_ID BUY_ORDER_ID SELL_ORDER_ID        DELETED_TIME  TICK_STATUS  OMDSEQ
0 2022-03-02 10:00:07.592961899 1969-12-31 19:00:00    NaN     1                       LEG            -         1                                     1969-12-31 19:00:00            0       9
1 2022-03-02 10:00:07.593666573 1969-12-31 19:00:00    NaN     2                       LEG            -         1                                     1969-12-31 19:00:00            0       0
2 2022-03-02 10:02:20.046438331 1969-12-31 19:00:00    NaN     3                       LEG            -         1                                     1969-12-31 19:00:00            0       0
3 2022-03-02 10:03:15.584272405 1969-12-31 19:00:00    NaN     1                       LEG            -         1                                     1969-12-31 19:00:00            0       3
4 2022-03-02 10:03:35.612187685 1969-12-31 19

Bad:

In [56]:
trd = otp.DataSource('CME', tick_type='TRD')
trd_df = otp.run(trd, start=start, end=end, symbols=emini)
trd_df['TRADE_TYPE'].unique()

array(['-', 'LEG', 'IMP'], dtype=object)

Good:

In [57]:
trd = otp.DataSource('CME', tick_type='TRD')
trd = trd.distinct(keys=['TRADE_TYPE'])
otp.run(trd, start=start, end=end, symbols=emini)

,Time,TRADE_TYPE
0,2022-03-02 16:00:00,-
1,2022-03-02 16:00:00,LEG
2,2022-03-02 16:00:00,IMP


# More advanced concepts

### Timezones

In [58]:
# this sets the timezone for all of the otp calls until its redefined
otp.config['tz'] = 'GMT'
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
print(otp.run(trd.first()[['PRICE']],start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15)))

otp.config['tz'] = 'America/Chicago'
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
print(otp.run(trd.first()[['PRICE']],start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15)))

otp.config['tz'] = 'EST5EDT'
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
print(otp.run(trd.first()[['PRICE']],start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15)))


                           Time   PRICE
0 2022-03-01 10:00:04.133843068  164.01
                           Time     PRICE
0 2022-03-01 10:00:00.010168563  165.2822
                           Time   PRICE
0 2022-03-01 10:00:00.003529175  165.75


In [59]:
# you can also pass a timezone to otp.run()
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD', start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))
print(otp.run(trd,timezone='GMT').head())

trd = otp.DataSource('NYSE_TAQ', tick_type='TRD', start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))
print(otp.run(trd,timezone='America/Chicago').head())

                           Time EXCHANGE  COND STOP_STOCK SOURCE TRF TTE TICKER   PRICE DELETED_TIME  TICK_STATUS  SIZE  CORR  SEQ_NUM TRADE_ID              PARTICIPANT_TIME   TRF_TIME  OMDSEQ
0 2022-03-01 10:00:04.133843068        P  @ TI                 N       0   AAPL  164.01   1970-01-01            0     1     0    18621      452 2022-03-01 10:00:04.133499136 1970-01-01       0
1 2022-03-01 10:00:08.519225755        P  @ TI                 N       0   AAPL  164.00   1970-01-01            0     1     0    18635      453 2022-03-01 10:00:08.518881024 1970-01-01       0
2 2022-03-01 10:00:15.746801681        Q  @FTI                 N       1   AAPL  164.01   1970-01-01            0     1     0    18649      405 2022-03-01 10:00:15.746784019 1970-01-01       0
3 2022-03-01 10:00:30.490236820        P  @ TI                 N       0   AAPL  164.00   1970-01-01            0     1     0    18686      454 2022-03-01 10:00:30.489892608 1970-01-01       0
4 2022-03-01 10:00:30.497604203    

## Reverting to OneTick when otp is not enough: the sink() method
We are working to extend otp to cover most of the cases but OneTick is big and powerful and oftentimes it offers functionality that is not yet available in otp. The 'sink()' method lets you (pretty seamlessly) transition to a lower level API called onetick.query that supports all of OneTick features. See docs/onetick.query.api.python.html in OneTick documentation.

In [60]:
import onetick.query as otq
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')

trd.sink(otq.Variance(input_field_name='PRICE', output_field_name='VAR_PRICE', bucket_interval=60))
trd = trd.table(VAR_PRICE=float, strict=False) # need to explicitly add the fields added by the `sink` methods to the schema

trd['std'] = otp.math.sqrt(trd['VAR_PRICE'])

otp.run(trd,  start=otp.dt(2022,9,1,10), end=otp.dt(2022,9,1,16), symbols='AA')

,Time,VAR_PRICE,std
0,2022-09-01 10:01:00,0.016861,0.129851
1,2022-09-01 10:02:00,0.001083,0.032903
2,2022-09-01 10:03:00,0.004457,0.066762
3,2022-09-01 10:04:00,0.001522,0.039015
4,2022-09-01 10:05:00,0.001840,0.042895
...,...,...,...
355,2022-09-01 15:56:00,0.000230,0.015159
356,2022-09-01 15:57:00,0.000491,0.022159
357,2022-09-01 15:58:00,0.000508,0.022529
358,2022-09-01 15:59:00,0.000034,0.005794


## Filters: using both `if` and `else` branches
Filters return two objects: one for the ticks that satisfy the conditions and the other one for the ones that don't. Having both objects allows combining the two streams without duplication anything that happens before the filter. An example is below.

In [61]:
# this processing happens before a filter
trd = otp.DataSource('NYSE_TAQ', tick_type='TRD')
trd = trd[['PRICE','SIZE','EXCHANGE','COND']]
trd = trd.first(5)
print(otp.run(trd, symbols='AAPL', start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15)))

# the filter is applied
q, non_q = trd[trd['EXCHANGE']=='Q']

# different processing can be applied to the different braches of the filter
q = q.first()

# the two branches can them be merged
merged = otp.merge([q, non_q])

otp.run(merged, symbols='AAPL', start=otp.dt(2022,3,1,10), end=otp.dt(2022,3,1,10,15))

                           Time   PRICE  SIZE EXCHANGE  COND
0 2022-03-01 10:00:00.003529175  165.75   100        Q  @   
1 2022-03-01 10:00:00.003530928  165.75   100        Q  @   
2 2022-03-01 10:00:00.003532720  165.75   100        Q  @   
3 2022-03-01 10:00:00.003676379  165.75   100        Z  @   
4 2022-03-01 10:00:00.004032611  165.75   104        H  @F  


,Time,PRICE,SIZE,EXCHANGE,COND
0,2022-03-01 10:00:00.003529175,165.75,100,Q,@
1,2022-03-01 10:00:00.003676379,165.75,100,Z,@
2,2022-03-01 10:00:00.004032611,165.75,104,H,@F
